In [152]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import Chrome, ChromeOptions
from selenium.common.exceptions import NoSuchElementException
import time
import random
from tqdm import notebook
import pandas as pd
from datetime import datetime

In [153]:
# 所有需抓取的shop_id列表
data = pd.read_csv('shopid.csv')
shop_url_list = data['id'].unique()
shop_url = shop_url_list[0]

In [154]:
# 店铺url
basic_url = 'http://www.dianping.com/shop/'

In [155]:
yz_err_list = [] # 因为验证未通过无法抓取的id
page_err_list = [] # 因为网页结构问题无法抓取的id

In [ ]:
scraped = []
for i in [csv for csv in os.listdir() if csv.find('tempsave') != -1]: # 所有已被抓取的都被存入以tempsave开头的csv文件中
    scraped += list(pd.read_csv(i)['id'].unique())
scraped_set = set(scraped)

In [ ]:
# '''
# Input: 
# remain_set: ids to be scraped
# proxy_num: the port that mitmdump is listening to 
# webdriver_path: where your webdriver launcher is 
# '''

remain_set = set(shop_url_list) - scraped_set - set(yz_err_list) - set(page_err_list)
print(len(remain_set)) # 需被抓取的url的数量
proxy_num = '8080'
webdriver_path = '/webdriver_path'


In [ ]:

scraped_list = []
err_list = []

# set Chromedriver options to conceal webdriver feature
option = ChromeOptions()
option.add_experimental_option('useAutomationExtension', False)
option.add_experimental_option('excludeSwitches', ['enable-automation'])
option.add_argument("--disable-blink-features")
option.add_argument("--disable-blink-features=AutomationControlled")
PROXY = f"127.0.0.1:{proxy_num}"
option.add_argument('--proxy-server=%s'%PROXY)
driver = webdriver.Chrome(webdriver_path
                        , options=option)


for shop_url_idx in notebook.tqdm(range(len(remain_set))):
    try:
        # save every 150 ids
        if (shop_url_idx%150 == 0) & (shop_url_idx != 0):
            temp_save = pd.DataFrame(scraped_list, columns = ['id', 'comment_text'])
            temp_save.to_csv(f'tempsave_{str(datetime.now())}.csv', index = False)
            print(f'saved to tempsave_{str(datetime.now())}.csv')
            time.sleep(50)
        
        url1 = basic_url + list(remain_set)[shop_url_idx]        
        contents = driver.get(url1)
        time.sleep(random.randint(8, 12))
        
        # id == summaryfilter-wrapper, for comments
        res = driver.find_element_by_id('summaryfilter-wrapper').text
        scraped_list.append((list(remain_set)[shop_url_idx], res))

    except NoSuchElementException:
        print(url1)
        # 每次保存都存一下以防万一
        temp_save = pd.DataFrame(scraped_list, columns = ['id', 'comment_text'])
        temp_save.to_csv(f'tempsave{str(datetime.now())}.csv', index = False)
        while True: 
            if "验证中心" in driver.page_source:
                yz_err_list.append(list(remain_set)[shop_url_idx])
                time.sleep(10)
            else:
                break
        if "验证中心" not in driver.page_source:
            try:
                res = driver.find_element_by_id('summaryfilter-wrapper').text
                scraped_list.append((list(remain_set)[shop_url_idx], res))
                time.sleep(8)
            except NoSuchElementException:
                page_err_list.append(list(remain_set)[shop_url_idx])
                time.sleep(4)
        pass

